# Module 5: Matrix Factorization

In the previous module, you calculated a utility matrix with predicted ratings, given a user matrix and a movie matrix with scores for specific genres.

Now, we are going to do the opposite. We will determine the user and movie matrices based on the given ratings (the `target` matrix from the end of previous module). This process is called _matrix factorization_.

Matrix factorization is quite similar to integer factorization. A question you might have encountered before: If $a \times b = c$ and we know that $c = 9$, what are $a$ and $b$?

Finding an answer to this question is called factoring $c$. This is very similar to factoring matrices. In our case, the question is: Given $M \cdot U^T = R$, and we know what $R$ is (the `target` matrix), what are the factors $M$ and $U$?

One significant difference (compared to integer factorization) is that matrix factorization often doesn't have an exact answer. So the question becomes: Given $M \cdot U^T = \hat{R}$, what are the values for $M$ and $U$ so that $\hat{R}$ closely approximates $R$? In this case, we use the MSE to determine how close we are to the correct answer.

There are multiple ways to factorize matrices. Some of these methods require a good deal of knowledge of linear algebra. We won't be exploring those approaches here. However, it turns out that the algorithmic workhorse of machine learning, _gradient descent_, is very suitable for finding the matrix factors. The advantage of gradient descent is its intuitive nature (understandable without delving too deep into linear algebra). Furthermore, gradient descent is an incredibly versatile algorithm that is useful to know regardless.

Calculating the optimal $M$ and $U$ mathematically in one go can be quite challenging. There are many intricacies involved, and in some cases, it's simply not feasible. But it's relatively straightforward to find a mathematical way to make slight adjustments to $M$ and $U$ so that the prediction $\hat{R}$ becomes slightly better after each adjustment.

This is where gradient descent comes into play. The algorithm works roughly like this: Start by filling in random values for all the entries in matrices $M$ and $U$. Then, attempt to adjust $M$ and $U$ in small steps (updates), getting closer and closer to the answer.

This module consists of three parts:
- Part 1: _Gradient Descent_
- Part 2: _Alternating Least Squares_
- Part 3: Evaluation

The final part contains an open assignment, which will require more time to complete compared to the other assignments. Keep this in mind when planning your schedule.

Before you get started with gradient descent, first load the necessary packages and the data from the previous module below:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from helpers import nice, random_df, display_side_by_side
import answers

data_folder = './data/'
pre_defined_items = pd.read_csv(f'{data_folder}three_genres_movies.csv', index_col = 'movieName')
pre_defined_users = pd.read_csv(f'{data_folder}three_genres_users.csv', index_col = 'userId')
predicted = pre_defined_items @ pre_defined_users.T
ratings = pd.read_csv(f'{data_folder}three_genres_ratings.csv', index_col = 'movieName', dtype={'userId': np.int64})
target = ratings.pivot(columns = 'userId', values = 'value').loc[predicted.index, predicted.columns]

# display data
display(nice(pre_defined_items))
display(nice(target.iloc[:15,:15], nan = '&middot;'))

## Part 1: Gradient Descent

### Learning the User Matrix First

To keep the problem somewhat manageable, let's use the given movie matrix and first focus on calculating (learning) only the user matrix $U$. Later, we will explore how to learn both matrices simultaneously.

It's reasonable to assume that we know the movie matrix but not the user matrix: We assume that we know which genres the movies belong to, but we don't know the genre preferences of the users. We need to learn this information from the data. You might encounter similar situations in practical scenarios.

So, we want to figure out how to adjust the user matrix to make the predictions better. This involves having the predicted ratings, $\hat{R} = M \cdot U^T$, and wanting to understand how to modify $U$ to improve these predicted ratings, $\hat{R}$. It turns out that you can work this out mathematically. The derivation involves some calculus and goes beyond the scope of this assignment. However, below, we provide the equations you would arrive at if you worked out the mathematics.

The solution consists of two steps:

1. First, we need to determine how much we are off. We need to find the difference between the predicted rating and the target:

    $$
    D = R - \hat{R}
    $$

    <table style= "width:100%"><tr><td style="border:1px solid black">

    **Keep in mind that $R$ contains `nan` values.** If this is the case, performing this calculation in Pandas would result in `nan` values in the corresponding positions of the outcome $D$. This could cause issues in the calculation below. So, all `nan` values in $D$ need to be replaced with zeros.

    </td></tr></table>

2. Next, we want to know how to update $U$ to make that difference $D$ slightly smaller. The update (a step) for the user matrix looks like this:

    $$
    U := U + \alpha \cdot D^T \cdot M
    $$

    Here, $\alpha$ is not a matrix but just a number. That number is the _learning rate_, a small value that determines the size of the steps you take when updating the matrices. If you choose $\alpha$ to be too large, learning might not work well. Choosing it too small, however, will lead to learning very slowly. Often, finding a good value requires trial and error. For these assignments, we'll start with setting the value of $\alpha$ to $0.04$.

    Updating the user matrix $U$ involves the matrix product $D^T \cdot M$. So, you'll need the movie matrix $M$ to calculate the new $U$.

**If we use this update and recalculate $\hat{R} = M \cdot U^T$ after the update, the mean squared error (i.e., $\textrm{MSE}(R, \hat{R})$) after the update should be slightly lower than before the update.**

### Gradient Descent in Pseudocode

The above two equations are sufficient for gradient descent. Given the target $R$ and the given movie matrix $M$, the algorithm looks like this:

- Generate a random user matrix, $U$.
- Repeat $N$ iterations:
    1. Calculate the utility matrix (predict ratings), $\hat{R} = M \cdot U^T$.
    2. Compute the MSE based on the current prediction (the _cost_): $\textrm{cost} = \textrm{mse}(R, \hat{R})$.
    3. Update the user matrix:
    $$
    D = R - \hat{R}
    $$
    $$
    U := U + \alpha \cdot D^T \cdot M
    $$

You'll determine good-working values for $\alpha$ and $N$ experimentally.

### Question 1

\[2 pt.\]

The majority of the algorithm is already implemented below. The only thing missing is the mathematics from step 3 for updating the user matrix.

Implement the function `update_users` below. This function takes as input the movie matrix (`movies`), the current user matrix (`users`), the prediction as calculated in step 1 of the algorithm (`prediction`), the target matrix (`target`), and the learning rate (`alpha`). The function should compute the new user matrix using the equations provided in step 3 of the algorithm.


<table style= "width:100%"><tr><td style="border:1px solid black">

Tip: Don't forget that $R$ contains a lot of `nan` values. This means that $D$ in the equations above will also contain many `nan` values. These values are problematic for the rest of the algorithm. Therefore, it's important to replace the `nan` values in $D$ with $0$.
    
</td></tr></table>

In [ ]:
def MSE(m1: pd.DataFrame, m2: pd.DataFrame) -> float:
    se = (m2 - m1)**2
    return se.mean().mean()

def update_users(movies: pd.DataFrame, users: pd.DataFrame, prediction: pd.DataFrame,
                 target: pd.DataFrame, alpha: float) -> pd.DataFrame:
    # TODO
    


def fit_user(target: pd.DataFrame, movies: pd.DataFrame, 
             iterations: int = 200, alpha: float = 0.04) -> tuple[pd.DataFrame, list]:
    # compute random user matrix
    users = random_df(target.columns, movies.columns)

    costs = []

    # Repeat N times
    for i in tqdm(range(iterations)):
        # 1. Predict utility
        prediction = movies @ users.T

        # 2. Compute cost (mse)
        cost = MSE(target, prediction)

        # 3. Update user matrix
        users = update_users(movies, users, prediction, target, alpha)

        costs.append(cost)
    
    
    return users, costs
    
users, costs = fit_user(target, pre_defined_items, iterations = 100, alpha = 0.04)
    
print(f'The mse of the final prediction is {costs[-1]:.5f}')
plt.semilogy(costs)
plt.xlabel('iteration')
plt.ylabel('cost (mse)')
plt.show()

If you have implemented the function correctly, you should see in the plot that the mean squared error (MSE) decreases slightly with each iteration.

Please verify if your implementation seems correct:

In [ ]:
answers.test_1(costs)

With the code below, you can compare parts of $R$ and $\hat{R}$ side by side. Do the predicted values from $\hat{R}$ match the provided values from $R$?

You'll notice there are many `nan` values in $R$, where $\hat{R}$ makes predictions. These are the exact predictions we can ultimately use when making new recommendations.

In [ ]:
display_side_by_side(nice(target.iloc[:16,:4], nan = '&middot;'), 
                     nice((pre_defined_items @ users.T).iloc[:16,:4]), 
                     titles = ["target", "learned"])

You can run the algorithm for more iterations to ensure the MSE becomes even smaller. When increasing the learning rate $\alpha$, the learning process happens quicker, requiring fewer iterations to achieve a lower MSE score. However, there is a limit to this; setting the learning rate too high might cause the algorithm to stop working correctly.

### Question 2

\[1 pt.\]

Experiment below with the parameters (`iterations` and `alpha`) in the `fit_user` function call. Ensure that the MSE eventually goes below $0.005$.

<table style= "width:100%"><tr><td style="border:1px solid black">

Tip: Each time you run the algorithm, the outcome might vary slightly. Once you find good parameter values, run the algorithm a few more times with the same values to ensure it consistently results in an MSE below $0.005$.
  
</td></tr></table>

In [ ]:
# TODO


# users, costs = fit_user(target, pre_defined_items, iterations = ???, alpha = ???)

print(f'The mse of the final prediction is {costs[-1]:.5f}')
plt.semilogy(costs)
plt.xlabel('iteration')
plt.ylabel('cost (mse)')
plt.show()

Again, compare slices of $R$ and $\hat{R}$ below. Does $\hat{R}$ appear to be better than the same matrix in question 1?

In [ ]:
display_side_by_side(nice(target.iloc[:16,:4], nan = '&middot;'), 
                     nice((pre_defined_items @ users.T).iloc[:16,:4]), 
                     titles = ["target", "learned"])

Test your implementation:

In [ ]:
answers.test_2(costs)

### Question 3

\[1 pt.\]

What is the maximum value for the learning rate $\alpha$ at which the algorithm still functions correctly?

YOUR ANSWER HERE

## Part 2: Alternating Least Squares

In the previous algorithm, we assumed knowing the genre scores for all movies. However, we often don't have that kind of information. Moreover, genre categorization might not always be the best way to characterize movies: Using genres like `drama`, `thriller`, and `comedy` might not provide the optimal representation of movie taste.

We can solve these issues by learning not just the user matrix, but also the movie matrix. For this, we can use almost the same algorithm as above. The primary adjustment lies in the update step. In addition to an update for $U$, we need an update for $M$. The update (one step) for the matrices looks like this:

$$
M := M + \alpha \cdot D \cdot U
$$
$$
U := U + \alpha \cdot D^T \cdot M
$$

We have added an update for $M$ here. The update for $U$ remains the same as before.

However, there is a slight extra complication here: The update of the user matrix depends on the movie matrix, and vice versa. This means we can't simply update them simultaneously.

The trick is to update them not _simultaneously_ but rather _alternately_: First, we assume that the movie matrix doesn't change and we update only the user matrix (similar to what we did in the previous assignment). Then, we reverse the roles, assuming that the user matrix doesn't change, and update the movie matrix.

The algorithm looks like this (given the target matrix $R$):

- Generate a random user matrix, $U$, and a random movie matrix $M$.
- Repeat N iterations:
    - Phase 1 (update user matrix)
        1. Calculate the utility matrix (predict ratings), $\hat{R} = M \cdot U^T$
        2. Calculate the MSE based on the current prediction (the _cost_): $\textrm{cost} = \textrm{mse}(R, \hat{R})$
        3. Update the user matrix:
        $$
        D = R - \hat{R}
        $$
        $$
        U := U + \alpha \cdot D^T \cdot M
        $$
    - Phase 2 (update movie matrix)
        4. Calculate the utility matrix (predict ratings), $\hat{R} = M \cdot U^T$
        5. Calculate the MSE based on the current prediction (the _cost_): $\textrm{cost} = \textrm{mse}(R, \hat{R})$
        6. Update the movie matrix:
        $$
        D = R - \hat{R}
        $$
        $$
        M := M + \alpha \cdot D \cdot U
        $$

Due to the alternating roles of the user and movie matrices, we call this an *Alternating Least Squares* (ALS) algorithm.

It's important to realize that by using ALS, we have also let go of the idea of movie genres. We are learning both the user and movie matrices. Since we provide random initial values for both, we lost specific information about movie genres embedded in them.

So what do the scores end up meaning? It's difficult to say. It's a categorization that best predicts the data. It might be that a category leads to high scores for action movies, movies featuring Brad Pitt, or movies shorter than 100 minutes. Alternatively, it could be a more abstract categorization that we can't meaningfully interpret. However, this doesn't matter as long as it produces accurate predictions.

The only thing we can say is that two movies with similar scores for different categories are similar. They are similar in the sense that if a user likes one movie, there's a good chance they'll also like the other.

So, we're not just learning the best scores for movie categories, but also the best categories to describe a movie with.

### Question 4

\[3 pt.\]

The alternating least squares algorithm is partially implemented below. Phase 2 of the algorithm is not, and the `update_movies` function is incomplete too. Implement these missing parts.

In [ ]:
def update_movies(movies: pd.DataFrame, users: pd.DataFrame, prediction: pd.DataFrame,
                  target: pd.DataFrame, alpha: float) -> pd.DataFrame:
    # TODO
    


def fit(target: pd.DataFrame, iterations: int = 200, alpha: float = 0.04,
        number_of_categories: int = 3) -> tuple[pd.DataFrame, pd.DataFrame, list]:
    # compute random matrices
    users = random_df(target.columns, list(range(number_of_categories))).copy()
    movies = random_df(target.index, list(range(number_of_categories))).copy()

    costs = []

    # repeat N times
    for i in tqdm(range(iterations//2)):
        # phase 1 (update user matrix)
        
        # 1. predict utility
        prediction = movies @ users.T

        # 2. compute cost (mse)
        cost = MSE(target, prediction)
        costs.append(cost)

        # 3. update user matrix
        users = update_users(movies, users, prediction, target, alpha)

        # phase 2 (update item matrix)
 
        # TODO
        
    
    return users, movies, costs
    
users, movies, costs = fit(target, iterations = 100, alpha = 0.04)
    
print(f'The mse of the final prediction is {costs[-1]:.5f}')
plt.semilogy(costs)
plt.xlabel('iteration')
plt.ylabel('cost (mse)')
plt.show()

Test your implementation:

In [ ]:
answers.test_4(costs)

### Question 5

\[1 pt.\]

Experiment with the parameters (`iterations` and `alpha`) below. Ensure that the MSE eventually goes below $0.001$.

In [ ]:
# TODO



print(f'The mse of the final prediction is {costs[-1]:.5f}')
plt.semilogy(costs)
plt.xlabel('iteration')
plt.ylabel('cost (mse)')
plt.show()

Test your implementation:

In [ ]:
answers.test_5(costs)

### Question 6

\[2 pt.\]

As mentioned, the columns in the user and movie matrix no longer correspond to specific genres, as was the case in the first part of this module. There is also no clear reason to keep using only 3 categories (columns). We could have 4 categories, 2 categories, or 10 categories in the matrices, as long as the number of columns is the same for both matrices. Adding more categories allows us to encode more information (although it doesn't necessarily lead to better predictions).

The `fit` function generates matrices with 3 categories by default, but it has a parameter, `number_of_categories`, that allows you to adjust this number. Below, run the `fit` function for 2, 3, and 4 categories with `iterations = 300` and `alpha = 0.1`.

Plot the `costs` of all three runs in one graph. Make sure the graph is clearly labeled (use a legend).

In [ ]:
# TODO


## Part 3: Evaluation

You now know how to factorize a matrix using the alternating least squares algorithm. This is a universal algorithm, which can be used for much more than just recommender systems. Whenever you have a matrix with missing values (NaN values), theoretically, you can use matrix factorization to predict these values.

There's one important aspect we haven't covered in this assignment: we haven't evaluated the results of this matrix factorization. Although you calculated the MSE, it hasn't been done on a separate test set. As you've seen in previous assignments, we need to use a test set for proper evaluation. The use of MSE in this assignment was **not** for evaluating the algorithm. We used it solely for updating the matrices and monitoring the progress of the algorithm.

However, you can apply the same evaluation techniques as in the previous modules here as well. Just like in the previous modules, you predict ratings for movies and users. With these predictions, you can recommend movies (by choosing a threshold value, for instance), and these recommendations can be used for evaluation using metrics like *precision* and *recall*.

### Question 7

\[4 pts.\] (execution: 2 pts, conclusion: 2 pts.)

For this assignment, you will work with the dataset from module 2. You can use the following commands to load the data:

    training_data = pd.read_pickle('./mini-movielens/ratings_t80_training.pkl')
    test_data = pd.read_pickle('./mini-movielens/ratings_t80_test.pkl')

What we would like to know is: What is the best number of categories for this data? You can use the MSE of the predictions against the _test data_ to determine this. In concrete terms, the question you need to answer is:

**Which number of categories results in the lowest MSE for the test data?** Create plots to support your answer.

It's up to you to figure out how to approach this. What will you plot? How will you transform the data? How will you determine a good learning rate? How many iterations will you perform? Write your conclusion in the next cell. Write your code in the cell below.

In your conclusion, mention the best number of categories and describe how you deduced it from the generated plots. If you want to modify the `fit` function, do not modify the code above. If you want to make adjustments to the `fit` function (or any other function you've written above), copy it into the code cell below and make changes there.

YOUR ANSWER HERE

In [ ]:
# TODO
